# Prueba de detección de distancias
Esto detecta los links a ver si existen y te dice el elemento más próximo

In [2]:
import math

def calculate_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def main(file_path):
    elements = []
    links = []

    # Leer el archivo
    with open(file_path, 'r') as file:
        for line in file:
            data = line.strip().split()
            class_id = int(data[0])
            center_x = float(data[1])
            center_y = float(data[2])
            width = float(data[3])
            height = float(data[4])

            if class_id == 1:  # Identificador de Link
                links.append({'x': center_x, 'y': center_y, 'width': width, 'height': height})
            else:
                element_type = {0: 'DNS', 2: 'Router', 3: 'Switch'}[class_id]
                elements.append({'id': element_type, 'x': center_x, 'y': center_y, 'width': width, 'height': height})

    # Calcular conexiones
    connections = {}
    for i, element in enumerate(elements):
        element_id = f"{element['id']}_{i+1}"
        min_distance = float('inf')
        closest_link = None
        
        for link in links:
            distance = calculate_distance(element['x'], element['y'], link['x'], link['y'])
            if distance < min_distance:
                min_distance = distance
                closest_link = link

        connections[element_id] = {'connected_to': closest_link, 'distance': min_distance}

    # Imprimir las conexiones
    for key, value in connections.items():
        print(f"{key} is connected to a Link with distance {value['distance']}")

file_path = './1.txt'
main(file_path)


Router_1 is connected to a Link with distance 0.14056075573217436
DNS_2 is connected to a Link with distance 0.17658045274605005
DNS_3 is connected to a Link with distance 0.20606288224714325
DNS_4 is connected to a Link with distance 0.20744373067653799
DNS_5 is connected to a Link with distance 0.14825935558001055
Switch_6 is connected to a Link with distance 0.18519025806451053


# Conexión entre elementos suprimiendo links
Esto formará un esquema básico suprimiendo los links, esto corrije de forma autonoma la posibilidad de que los links no existan

In [1]:
import math

elements = []
def calculate_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def main(file_path):

    # Leer el archivo
    with open(file_path, 'r') as file:
        for line in file:
            data = line.strip().split()
            class_id = int(data[0])
            if class_id != 1:  # Ignoramos los links
                element_type = {0: 'DNS', 2: 'Router', 3: 'Switch'}[class_id]
                center_x = float(data[1])
                center_y = float(data[2])
                elements.append({'id': element_type, 'x': center_x, 'y': center_y})

    # Función para encontrar el elemento más cercano
    def find_closest_element(current, others):
        min_distance = float('inf')
        closest_element = None
        for element in others:
            distance = calculate_distance(current['x'], current['y'], element['x'], element['y'])
            if distance < min_distance:
                min_distance = distance
                closest_element = element
        return closest_element, min_distance

    # Calcular conexiones directas
    connections = {}
    for i, element in enumerate(elements):
        current_id = f"{element['id']}_{i+1}"
        others = [e for j, e in enumerate(elements) if i != j]
        closest, distance = find_closest_element(element, others)
        if closest:
            closest_id = f"{closest['id']}_{elements.index(closest)+1}"
        connections[current_id] = {'connected_to': closest_id, 'distance': distance}

    # Imprimir las conexiones
    for key, value in connections.items():
        print(f"{key} is connected to {value['connected_to']} with distance {value['distance']}")
    
file_path = './1.txt'
main(file_path)


Router_1 is connected to Switch_6 with distance 0.2806729552949481
DNS_2 is connected to Router_1 with distance 0.33973900160270093
DNS_3 is connected to DNS_4 with distance 0.36645383081637994
DNS_4 is connected to DNS_3 with distance 0.36645383081637994
DNS_5 is connected to Router_1 with distance 0.2887971355900193
Switch_6 is connected to Router_1 with distance 0.2806729552949481


In [4]:
import json
import uuid

# Función para generar una estructura de nodo basada en los datos del elemento y su tipo
def generate_node(element_id, node_type, x, y, width, height, adapters):
    node_id = str(uuid.uuid4())
    return {
        "compute_id": "local",
        "console": 5000,
        "console_auto_start": True,
        "console_type": "telnet",
        "custom_adapters": [],
        "first_port_name": None,
        "height": height,
        "label": {
            "rotation": 0,
            "style": "font-family: TypeWriter;font-size: 10.0;font-weight: bold;fill: #000000;fill-opacity: 1.0;",
            "text": element_id,
            "x": 0,
            "y": -25
        },
        "locked": True,
        "name": element_id,
        "node_id": node_id,
        "node_type": node_type,
        "port_name_format": "Ethernet{0}",
        "port_segment_size": 0,
        "properties": {
            "adapters": adapters,
            "aux": 5001,
            "console_http_path": "/",
            "console_http_port": 80,
            "console_resolution": "800x600",
            "container_id": str(uuid.uuid4()),
            "environment": f"MAC=0\nTYPE={adapters}",
            "extra_hosts": None,
            "extra_volumes": [],
            "image": "jcfabero/server:latest",
            "start_command": "/bin/bash",
            "usage": "Configure network settings."
        },
        "symbol": ":/symbols/classic/server.svg",
        "template_id": str(uuid.uuid4()),
        "width": width,
        "x": x,
        "y": y,
        "z": 1
    }

# Definir datos generales del proyecto
project_data = {
    "auto_close": True,
    "auto_open": False,
    "auto_start": False,
    "drawing_grid_size": 25,
    "grid_size": 75,
    "name": "1",
    "project_id": str(uuid.uuid4()),
    "revision": 9,
    "scene_height": 1000,
    "scene_width": 2000,
    "show_grid": False,
    "show_interface_labels": False,
    "show_layers": False,
    "snap_to_grid": False,
    "supplier": None,
    "topology": {
        "computes": [],
        "drawings": [],
        "links": [],
        "nodes": []
    },
    "type": "topology",
    "variables": None,
    "version": "2.2.29",
    "zoom": 100
}

# Añadir nodos basados en los datos de conexión
element_types = {'DNS': 'server', 'Router': 'router', 'Switch': 'switch'}
for element in elements:
    element_id = f"{element['id']}_{elements.index(element)+1}"
    node_type = element_types[element['id']]
    x = int(element['x'] * project_data['scene_width']) - 500
    y = int(element['y'] * project_data['scene_height']) - 500
    node_data = generate_node(element_id, node_type, x, y, 70, 45, 1)
    project_data['topology']['nodes'].append(node_data)

# Generar archivo JSON para el archivo .gns3
json_data = json.dumps(project_data, indent=4)

# Guardar el archivo JSON
output_file_path = './generated_topology.gns3'
with open(output_file_path, 'w') as file:
    file.write(json_data)

output_file_path


'./generated_topology.gns3'

PABLO

In [132]:
import math

def calculate_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

# Función para encontrar el elemento más cercano
def find_closest_element(current, others):
    min_distance = float('inf')
    closest_element = None
    for element in others:
        distance = calculate_distance(current['x'], current['y'], element['x'], element['y'])
        if distance < min_distance:
            min_distance = distance
            closest_element = element
    return closest_element, min_distance

def obtener_conexiones_validas(conexiones):
    # Creamos un conjunto vacío para almacenar las conexiones válidas
    conexiones_validas = set()

    # Iteramos sobre cada conexión
    for nodo, info in conexiones.items():
        nodo_a = nodo
        nodo_b = info['connected_to']

        # Verificamos si la conexión inversa existe
        if conexiones.get(nodo_b, {}).get('connected_to') == nodo_a:
            # Verificamos si ambos nodos están conectados al mismo link
            if conexiones.get(nodo_a, {}).get('distance') == conexiones.get(nodo_b, {}).get('distance'):
                conexiones_validas.add((nodo_a, nodo_b))
                conexiones_validas.add((nodo_b, nodo_a))

    return conexiones_validas

def main(file_path):
    elements = []
    links = []

    final_connections = {}

    # Leer el archivo
    with open(file_path, 'r') as file:
        for i, line in enumerate(file):
            data = line.strip().split()
            class_id = int(data[0])
            center_x = float(data[1])
            center_y = float(data[2])
            width = float(data[3])
            height = float(data[4])

            if class_id == 1:  # Identificador de Link
                links.append({'id': f"Link_{i+1}", 'x': center_x, 'y': center_y, 'width': width, 'height': height})
            else:
                element_type = {0: 'DNS', 2: 'Router', 3: 'Switch'}[class_id]
                elements.append({'id': f"{element_type}_{i+1}", 'x': center_x, 'y': center_y, 'width': width, 'height': height})

    # Calcular conexiones directas
    connections_to_elements = {}
    for i, element in enumerate(elements):
        current_id = f"{element['id']}"
        others = [e for j, e in enumerate(elements) if i != j]
        closest, distance = find_closest_element(element, others)
        if closest:
            closest_id = f"{closest['id']}"
        connections_to_elements[current_id] = {'connected_to': closest_id, 'distance': distance}

    # Calcular conexiones a links
    connections_to_links = {}
    for i, element in enumerate(elements):
        element_id = f"{element['id']}"
        min_distance = float('inf')
        closest_link = None
        
        for link in links:
            distance = calculate_distance(element['x'], element['y'], link['x'], link['y'])
            if distance < min_distance:
                min_distance = distance
                closest_link = f"{link['id']}"

        connections_to_links[element_id] = {'connected_to': closest_link, 'distance': min_distance}

    # Imprimir las conexiones
    for key, value in connections_to_elements.items():
        print(f"{key} <-> {value['connected_to']}")

    print()

    for key, value in connections_to_links.items():
        print(f"{key} <-> {value['connected_to']}")
        
    print('-----------------------------------------------------')
# PARA LAS CONEXIONES VALIDAS 1 HAY QUE MIRAR EN AMBOS NODOS SI SE CONECTAN AL MISMO LINK

    # Obtenemos las conexiones válidas
    conexiones_validas1 = obtener_conexiones_validas(connections_to_elements)
    conexiones_validas2 = obtener_conexiones_validas(connections_to_links)

    print("Conexiones válidas 1:")
    for conexion in conexiones_validas1:
        print(f"{conexion[0]} <-> {conexion[1]}")

    print("\nConexiones válidas 2:")
    for conexion in conexiones_validas2:
        print(f"{conexion[0]} <-> {conexion[1]}")

    print('-----------------------------------------------------')

    # for element in connections_to_elements:
    
    # print(final_connections)

file_path = './1.txt'
main(file_path)


Router_3 <-> Switch_8
DNS_4 <-> Router_3
DNS_5 <-> DNS_6
DNS_6 <-> DNS_5
DNS_7 <-> Router_3
Switch_8 <-> Router_3

Router_3 <-> Link_10
DNS_4 <-> Link_9
DNS_5 <-> Link_1
DNS_6 <-> Link_2
DNS_7 <-> Link_10
Switch_8 <-> Link_1
-----------------------------------------------------
Conexiones válidas 1:
Router_3 <-> Switch_8
DNS_5 <-> DNS_6
DNS_6 <-> DNS_5
Switch_8 <-> Router_3

Conexiones válidas 2:
-----------------------------------------------------


In [26]:
import math

def calculate_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def get_distances_dictionary(links, elements):
    
    distances = {}

    # Calcular distancia entre cada par de elementos
    for i, element1 in enumerate(elements):
        element1_id = f"{element1['id']}"
        distances[element1_id] = {}
        
        # Distancias a los elementos
        for j, element2 in enumerate(elements):
            if i != j:  # Evitar calcular la distancia entre el mismo elemento
                element2_id = f"{element2['id']}"
                distance = calculate_distance(element1['x'], element1['y'], element2['x'], element2['y'])
                distances[element1_id][element2_id] = distance

        # Distancias a los links
        for k, link in enumerate(links):
            link_id = f"{link['id']}"
            distance = calculate_distance(element1['x'], element1['y'], link['x'], link['y'])
            distances[element1_id][link_id] = distance

    return distances  
   
def get_distances_between_elements(elements):
    element_distances = {}
    for i, element_a in enumerate(elements):
        element_a_id = f"{element_a['id']}"
        element_distances[element_a_id] = {}
        for j, element_b in enumerate(elements):
            if i != j:
                element_b_id = f"{element_b['id']}"
                distance = calculate_distance(element_a['x'], element_a['y'], element_b['x'], element_b['y'])
                element_distances[element_a_id][element_b_id] = distance    
    
    for element, conexiones in element_distances.items():
        element_distances[element] = dict(sorted(conexiones.items(), key=lambda x: x[1]))
        
    return element_distances

def get_distances_between_elements_and_links(elements, links):
    element_link_distances = {}
    for element in elements:
        element_id = f"{element['id']}"
        element_link_distances[element_id] = {}
        for link in links:
            link_id = f"{link['id']}"
            distance = calculate_distance(element['x'], element['y'], link['x'], link['y'])
            element_link_distances[element_id][link_id] = distance
            
    
    for element, conexiones in element_link_distances.items():
        element_link_distances[element] = dict(sorted(conexiones.items(), key=lambda x: x[1]))
        
    return element_link_distances

def find_common_links(element_links, link_connections):
    # Inicializa un diccionario vacío para almacenar las conexiones
    connections = {}

    # Itera sobre cada elemento en element_links
    for element, links in element_links.items():
        # Verifica si el elemento está presente en link_connections
        if element in link_connections:
            # Itera sobre los enlaces del elemento
            for link, value in links.items():
                # Verifica si el enlace está presente en link_connections para el mismo elemento
                if link in link_connections[element]:
                    # Verifica si el valor del enlace coincide
                    if value == link_connections[element][link]:
                        # Almacena la conexión
                        connections[(element, link)] = value

    return connections

def main(file_path):
    "Calculamos la distancia entre el primer elemento del txt con el resto de elementos"
    elements = []
    links = []
    element_distances = {}
    element_link_distances = {}
    distances = {}

    # Leer el archivo
    with open(file_path, 'r') as file:
        for i, line in enumerate(file):
            data = line.strip().split()
            class_id = int(data[0])
            center_x = float(data[1])
            center_y = float(data[2])
            width = float(data[3])
            height = float(data[4])

            if class_id == 1:  # Identificador de Link
                links.append({'id': f"Link_{i+1}", 'x': center_x, 'y': center_y, 'width': width, 'height': height})
            else:
                element_type = {0: 'DNS', 2: 'Router', 3: 'Switch'}[class_id]
                elements.append({'id': f"{element_type}_{i+1}", 'x': center_x, 'y': center_y, 'width': width, 'height': height})

    element_distances= get_distances_between_elements(elements)
    element_link_distances= get_distances_between_elements_and_links(elements, links)
    
    #-------------------------
    # Encuentra conexiones basadas en los diccionarios proporcionados
    connections = find_common_links(element_distances, element_link_distances)

    # Imprime las conexiones
    print("Conexiones entre elementos:")
    for (element1, element2), value in connections.items():
        print(f"{element1} <-> {element2}: {value}")
    #-------------------------
'''
    # Pintar elemnto por fila
    for element_id_a, connections_a in element_distances.items():
        print(f"{element_id_a}:")
        for element_id_b, distance in connections_a.items():
            print(f"    {element_id_b}: {distance}")
            
    print('--------------------------------------------------------------------')

    for element_id_a, connections_a in element_link_distances.items():
        print(f"{element_id_a}:")
        for element_id_b, distance in connections_a.items():
            print(f"    {element_id_b}: {distance}")
    '''
            
    
    # distances= get_distances_dictionary(links, elements)

    # for element_id_a, connections_a in distances.items():
    #     for element_id_b, connections_b in distances.items():
    #         is_connected = calculate_connection(element_id_a, element_id_b, connections_a, connections_b)
    #         if is_connected:
    #             # guardamos conexion
                
    


file_path = './1.txt'
main(file_path)

Conexiones entre elementos:


In [32]:
# Los dos diccionarios que representan elementos y sus enlaces
element_links = {
    'Router_3': {'Switch_8': 0.2806729552949481, 'DNS_7': 0.2887971355900193, 'DNS_4': 0.33973900160270093, 'DNS_5': 0.6494607758717996, 'DNS_6': 0.6594596582809292},
    'DNS_4': {'Router_3': 0.33973900160270093, 'Switch_8': 0.4469826960095883, 'DNS_7': 0.6284019122066704, 'DNS_5': 0.6614758105403402, 'DNS_6': 0.8363767032348522},
    'DNS_5': {'DNS_6': 0.36645383081637994, 'Switch_8': 0.384698854100191, 'Router_3': 0.6494607758717996, 'DNS_4': 0.6614758105403402, 'DNS_7': 0.7778218644914785},
    'DNS_6': {'DNS_5': 0.36645383081637994, 'Switch_8': 0.4014254033341687, 'DNS_7': 0.6375325790930219, 'Router_3': 0.6594596582809292, 'DNS_4': 0.8363767032348522},
    'DNS_7': {'Router_3': 0.2887971355900193, 'Switch_8': 0.40515231583813016, 'DNS_4': 0.6284019122066704, 'DNS_6': 0.6375325790930219, 'DNS_5': 0.7778218644914785},
    'Switch_8': {'Router_3': 0.2806729552949481, 'DNS_5': 0.384698854100191, 'DNS_6': 0.4014254033341687, 'DNS_7': 0.40515231583813016, 'DNS_4': 0.4469826960095883},
}

link_connections = {
    'Router_3': {'Link_10': 0.14056075573217436, 'Link_9': 0.1631675684993804, 'Link_1': 0.44342334185290694, 'Link_2': 0.4528229828983948},
    'DNS_4': {'Link_9': 0.17658045274605005, 'Link_10': 0.4802593347036578, 'Link_1': 0.4957497933776675, 'Link_2': 0.6427832827765514},
    'DNS_5': {'Link_1': 0.20606288224714325, 'Link_2': 0.3471503732519958, 'Link_9': 0.6316562742647301, 'Link_10': 0.6984613586162086},
    'DNS_6': {'Link_2': 0.20744373067653799, 'Link_1': 0.36932660478227125, 'Link_10': 0.6308074353525013, 'Link_9': 0.7291330972394547},
    'DNS_7': {'Link_10': 0.14825935558001055, 'Link_9': 0.4519048328188137, 'Link_2': 0.4691058114178506, 'Link_1': 0.5887941860319275},
    'Switch_8': {'Link_1': 0.18519025806451053, 'Link_2': 0.19825377032732566, 'Link_10': 0.31379933175518393, 'Link_9': 0.32770925780026416},
}

# Inicializa un diccionario vacío para almacenar las conexiones
connections = {}

for element, links in element_links.items():
    for element2, distance in links.items():
        if element2 in link_connections[element]:
            if distance == link_connections[element][element2]:
                connections[(element, element2)] = distance
    
# Itera sobre cada elemento en element_links
for element, links in element_links.items():
    # Verifica si el elemento está presente en link_connections
    if element in link_connections:
        # Itera sobre los enlaces del elemento
        for link, value in links.items():
            # Verifica si el enlace está presente en link_connections para el mismo elemento
            if link in link_connections[element]:
                # Verifica si el valor del enlace coincide
                if value == link_connections[element][link]:
                    # Almacena la conexión
                    connections[(element, link)] = value

# Imprime las conexiones
print(connections)


{'Switch_8': 0.2806729552949481, 'DNS_7': 0.2887971355900193, 'DNS_4': 0.33973900160270093, 'DNS_5': 0.6494607758717996, 'DNS_6': 0.6594596582809292}
{'Router_3': 0.33973900160270093, 'Switch_8': 0.4469826960095883, 'DNS_7': 0.6284019122066704, 'DNS_5': 0.6614758105403402, 'DNS_6': 0.8363767032348522}
{'DNS_6': 0.36645383081637994, 'Switch_8': 0.384698854100191, 'Router_3': 0.6494607758717996, 'DNS_4': 0.6614758105403402, 'DNS_7': 0.7778218644914785}
{'DNS_5': 0.36645383081637994, 'Switch_8': 0.4014254033341687, 'DNS_7': 0.6375325790930219, 'Router_3': 0.6594596582809292, 'DNS_4': 0.8363767032348522}
{'Router_3': 0.2887971355900193, 'Switch_8': 0.40515231583813016, 'DNS_4': 0.6284019122066704, 'DNS_6': 0.6375325790930219, 'DNS_5': 0.7778218644914785}
{'Router_3': 0.2806729552949481, 'DNS_5': 0.384698854100191, 'DNS_6': 0.4014254033341687, 'DNS_7': 0.40515231583813016, 'DNS_4': 0.4469826960095883}
{}
